## Trabalho de Deep Learning

### Classificação de textos para análise de sentimentos

#### Aluno: Rafael Bertoldi Rossi

- O objetivo deste trabalho é criar um modelo binário de aprendizado de máquina para classificação de textos. 
Para isso, será utilizado a base de dados [IMDb](http://ai.stanford.edu/~amaas/data/sentiment/), que consiste de dados textuais de críticas positivas e negativas de filmes

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

## Import das bibliotecas

In [1]:
from torchtext.legacy import datasets
from torchtext.legacy import data
import torch
import torch.optim as optim
import torch.nn as nn
import random
import spacy
import pickle

## Carregamento da base de dados IMDB

Utilizando o spaCy Tokenizer - open source utilizado para tokenizar grandes quantidades de dados

In [2]:
torch.manual_seed(9876)
torch.backends.cudnn.deterministic = True

text = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

label = data.LabelField(dtype = torch.float)

Separação entre Treinamento, Validação e Teste

In [3]:
train, test = datasets.IMDB.splits(text, label)
train, validation = train.split(random_state = random.seed(9876))

Utilizando 10 mil 'word embeddings' pré-treinadas para otimizar o tempo de processamento e qualidade do modelo, já que haverão mais palavras com significados parecidos e uma conexão inicial já pré-estabelecida.

In [4]:
text.build_vocab(train, 
                 max_size = 10_000, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

label.build_vocab(train)

Configuração dos iterators e configurando para rodar em CPU e ordenação de cada batch através da sua dimensão.

In [5]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, validation, test), 
    batch_size=64,
    sort_within_batch=True,
    device = 'cpu')

## Criação do Modelo

Utilizada uma LSTM bidirecional, adicionando uma camada de dropout para evitar o overfitting. 256 neurônios na camada oculta e apenas 1 na saída por se tratar de um modelo de classificação binária. 

In [6]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu')) 
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)     
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
            
        return self.fc(hidden)

Inicialização do modelo

In [7]:
model = LSTM(vocab_size=len(text.vocab),
             embedding_dim=100,
             hidden_dim=64,
             output_dim=1,
             n_layers=2,
             bidirectional=True,
             dropout=0.2,
             pad_idx=text.vocab.stoi[text.pad_token])

Cópia das palavras pré-treinadas para a camdada de embedding.

In [8]:
model.embedding.weight.data.copy_(text.vocab.vectors)
model.embedding.weight.data[text.vocab.stoi[text.unk_token]] = torch.zeros(100)
model.embedding.weight.data[text.vocab.stoi[text.pad_token]] = torch.zeros(100)

## Treinamento do modelo

Configurando otimizador Adam por ser o mais recomendado devido aos excelentes resultados e critério de loss ideal para problemas de classificação binária.

In [9]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to('cpu')
criterion = criterion.to('cpu')

In [10]:
def binary_accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc


def train_model(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_model(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [11]:
num_epochs = 5
best_loss = 99999

In [12]:
for epoch in range(num_epochs):
    
    train_loss, train_acc = train_model(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_model(model, valid_iterator, criterion)
    
    # Salvar melhor estado do modelo
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(), 'melhor_modelo.pt')
        
    print('Epoch ', epoch)
    print('### Resultados na base Treino ###')
    print('Erro Treino:', train_loss)
    print('Acurácia Treino:', train_acc*100)
    print('')
    print('### Resultados na base Validação ###')
    print('Erro Validação:', valid_loss)
    print('Acurácia Validação:', valid_acc*100)
    print(50 * '-')

Epoch  0
### Resultados na base Treino ###
Erro Treino: 0.6441287733342526
Acurácia Treino: 61.914266156454154

### Resultados na base Validação ###
Erro Validação: 0.518243092096458
Acurácia Validação: 75.03972457627118
--------------------------------------------------
Epoch  1
### Resultados na base Treino ###
Erro Treino: 0.5191471068745982
Acurácia Treino: 74.92097888114678

### Resultados na base Validação ###
Erro Validação: 0.6089460319381649
Acurácia Validação: 71.1555438021482
--------------------------------------------------
Epoch  2
### Resultados na base Treino ###
Erro Treino: 0.512078031027404
Acurácia Treino: 75.2223996350365

### Resultados na base Validação ###
Erro Validação: 0.4884545214600482
Acurácia Validação: 76.48305084745762
--------------------------------------------------
Epoch  3
### Resultados na base Treino ###
Erro Treino: 0.40568940428486705
Acurácia Treino: 82.2992700729927

### Resultados na base Validação ###
Erro Validação: 0.42101279305199446
Acu

Salvar modelo

In [13]:
pickle.dump(model, open('model.pkl', 'wb'))

## Carregar modelo salvo e fazer previsão

In [14]:
model = pickle.load(open('model.pkl', 'rb'))

In [15]:
model.load_state_dict(torch.load('melhor_modelo.pt'))
test_loss, test_acc = evaluate_model(model, test_iterator, criterion)

print('### Resultados na base Treino ###')
print('Erro Treino:', test_loss)
print('Acurácia Treino:', test_acc*100)

### Resultados na base Treino ###
Erro Treino: 0.35210989831048817
Acurácia Treino: 85.43478261174448


In [16]:
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [token.text for token in nlp.tokenizer(sentence)]
    indexed = [text.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to('cpu')
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [17]:
predict_sentiment(model, "This movie is terrible")

0.980079710483551

In [18]:
predict_sentiment(model, "This movie is very good")

0.08272780478000641